In [1]:
from openai import OpenAI
from settings import config, system_message

import argparse
import pandas as pd
import json

from tqdm import tqdm
tqdm.pandas()

## Loading Sample Questions
The sample question CSV file has only five questions with the following columns:

* Question #,
* Statement,
* My Answer,
* Expected Answer,
* Answer Status,
* Comments,
* Categories,
* GCP Services or Tools,
* Concepts or Topics

After loading the data, apply a little data cleaning by replacing NaN and empty cells by #EMPTY# 

In [2]:
questions = pd.read_csv("sample_questions.csv", index_col=0, header=0)
questions = questions.fillna("#EMPTY#")
questions = questions.replace(["?","??","NaN"],"#EMPTY#")
questions.head()

,Statement,My Answers,Expected Answer,Answer Status,Comments,Categories,GCP Services or Tools,Concepts or Topics
Question #,,,,,,,,
51,You run a Cloud SQL instance for a business th...,B,B,Correct,\nA: Incorrect. Replication on Cloud SQL is ef...,5.5 Maintaining awareness of failures and\nmit...,#EMPTY#,#EMPTY#
52,You are running a Dataflow pipeline in product...,D,D,Correct,\nA: Incorrect. Re-reading the input data for ...,5.5 Maintaining awareness of failures and\nmit...,#EMPTY#,#EMPTY#
53,Your company is very serious about data protec...,B,C,Incorrect,\n,#EMPTY#,#EMPTY#,* Understand and Implement the principe of lea...
54,Cymbal Retail uses Google Cloud and has automa...,C,C,Correct,\n,#EMPTY#,#EMPTY#,#EMPTY#
55,Your company recently migrated to Google Cloud...,A,A,Correct,\n,#EMPTY#,#EMPTY#,#EMPTY#


## Generating prompt for openai
My prompt is composed of two parts: **system message** and **user message**

**System Messages**
I used the system message to describe what the model (GPT-4o) is supposed to do, how it should respond, and how it should categorize all the questions. For example: I'm requesting the model to give the answer in JSON format so I can parse it easily with Python's JSON library.

**User Message**
The user message is the question itself with all the additional data such as The expected answer, the initial categorization if exists, and a comment (I typically add comments for questions that I find difficult)

In [3]:
# Since the system message is lengthy, I decided to export its value to a setting file. Read the system message to understand what the system is supposed to do.
from settings import system_message
print(system_message)

You are an expert in Google Cloud Platform (GCP) and data engineering. 
Your task is to analyze the following question, which is intended for the Google Cloud Professional Data Engineering Exam Certification.

Please provide the output in clean JSON format with the following fields:

- **related_category:** [Identify all relevant categories from the list below. Include multiple categories if applicable.]
- **related_technologies:** [List all GCP services or other technologies explicitly mentioned or implied by the question.]
- **related_skills:** [Associate each skill directly with a service or technology from the `gcp_services_or_other_technologies` list. Focus on describing the specific skill or task related to
using that service or technology in the context of data engineering.]

### Categories:
- Data Access & Security
- Data Governance
- Data Ingestion
- Data Integration
- Data Lake
- Data Lakehouse
- Data Orchestration
- Data Processing
- Data Sharing & Transfer
- Data Visualizat

In [4]:
# Function to generate user messages for GPT-4o.
def generate_user_prompt(question):
    user_message = f"""
    \nQUESTION STATEMENT: {question["Statement"]}
    """

    if question["Expected Answer"] != "#EMPTY#":
        user_message += f"\nANSWER: {question['Expected Answer']}"

    if any([value != "#EMPTY#" for value in question[["Categories","GCP Services or Tools","Concepts or Topics"]].tolist()]):
        user_message += f"""\nINITIAL CATEGORIZATION: These are the categories that were initially set for the question below
        * **categories**: {question["Categories"]}
        * **gcp_services_or_tools**: {question["GCP Services or Tools"]}
        * **concepts**: {question["Concepts or Topics"]}
        """

    if question["Comments"] != "#EMPTY#":
        user_message += f"\nCOMMENTS: {question['Comments']}"
    return user_message

In [5]:
# example of user message
print(generate_user_prompt(questions.loc[51]))


    
QUESTION STATEMENT: You run a Cloud SQL instance for a business that requires that the database is accessible for transactions. You need to ensure minimal downtime for database transactions. What should you do?

A. Configure replication.
B. Configure high availability.
C. Configure backups.
D. Configure backups and increase the number of backups.
    
ANSWER: B
INITIAL CATEGORIZATION: These are the categories that were initially set for the question below
        * **categories**: 5.5 Maintaining awareness of failures and
mitigating impact
        * **gcp_services_or_tools**: #EMPTY#
        * **concepts**: #EMPTY#
        
COMMENTS: 
A: Incorrect. Replication on Cloud SQL is effective to offload reads. However, it does not support a full read-write database.
B: Correct. Configuring high availability on Cloud SQL will automatically switch to the secondary instance when the primary instance goes down, thus reducing downtime for the database's users.
C: Incorrect. Backups are usefu

## Call OpenAI to Categorize a Question

In [6]:
from settings import config

client = OpenAI(api_key=config.get("OPENAI_API_KEY"))

def categorize_question(question):
    """
    """
    user_message = generate_user_prompt(question)
    
    response = client.chat.completions.create(
        model = 'gpt-4o',
        messages = [
            {
              "role": "system",
              "content": [
                {
                  "type": "text",
                  "text": system_message
                }
              ]
            },
            {
              "role": "user",
              "content": [
                {
                  "type": "text",
                  "text": user_message
                }
              ]
            }
      ]
    )

    answer = response.choices[0].message.content
    return answer.replace("\n","").replace("```json","").replace("```","")

In [7]:
# example of categorization for gpt-4o
categorize_question(questions.loc[51])

'{  "related_categories": ["Data Access & Security", "Database Design"],  "related_technologies": ["Cloud SQL"],  "related_skills": [    "Cloud SQL: Configure high availability to minimize database downtime during failovers.",    "Cloud SQL: Enable automated backups for data recovery in case of data loss.",    "Cloud SQL: Use replication to offload read queries and improve performance."  ]}'

## Parse and save OpenAI response 

In [8]:
# Add categorization columns for generated data
questions['related_categories'] = ''
questions['related_technologies'] = ''
questions['related_skills'] = ''


def save_categorizations(question):
    """
    Parses the generated answer JSON and updates the gen_category, gen_services, and gen_concepts columns in the row.
    """
    # Call the generate_answer function with the question
    categorizations = categorize_question(question)

    try:
        json_categorizations = json.loads(categorizations)
        related_categories = "|".join(json_categorizations.get("related_categories", []))
        related_technologies = "|".join(json_categorizations.get("related_technologies", []))
        related_skills = "|".join(json_categorizations.get("related_skills", []))

        question['related_categories'] = related_categories
        question['related_technologies'] = related_technologies
        question['related_skills'] = related_skills
    except json.JSONDecodeError:
        print(f"Error parsing JSON for row: {question}")

    return question

In [9]:
# Apply the save_categorizations function to all questions of the dataset
questions = questions.progress_apply(save_categorizations, axis=1)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:07<00:00,  1.40s/it]


In [10]:
questions

,Statement,My Answers,Expected Answer,Answer Status,Comments,Categories,GCP Services or Tools,Concepts or Topics,related_categories,related_technologies,related_skills
Question #,,,,,,,,,,,
51,You run a Cloud SQL instance for a business th...,B,B,Correct,\nA: Incorrect. Replication on Cloud SQL is ef...,5.5 Maintaining awareness of failures and\nmit...,#EMPTY#,#EMPTY#,Database Design|Data Access & Security,Cloud SQL,Cloud SQL: Configure high availability to mini...
52,You are running a Dataflow pipeline in product...,D,D,Correct,\nA: Incorrect. Re-reading the input data for ...,5.5 Maintaining awareness of failures and\nmit...,#EMPTY#,#EMPTY#,Data Processing,Dataflow,Dataflow: Use side output to capture and separ...
53,Your company is very serious about data protec...,B,C,Incorrect,\n,#EMPTY#,#EMPTY#,* Understand and Implement the principe of lea...,Data Access & Security,Cloud IAM,Cloud IAM: Implement the Principle of Least Pr...
54,Cymbal Retail uses Google Cloud and has automa...,C,C,Correct,\n,#EMPTY#,#EMPTY#,#EMPTY#,Monitoring,Cloud Monitoring,Cloud Monitoring: Create alerts based on speci...
55,Your company recently migrated to Google Cloud...,A,A,Correct,\n,#EMPTY#,#EMPTY#,#EMPTY#,Data Warehousing|Data Access & Security,BigQuery|Cloud IAM,BigQuery: Understand and recommend the use of ...


The last three columns are the categorizations.

## Author
CARMEL WENGA - Machine Learning, Data Engineer, Software Engineer